# [SITCOM-1171] - Investigate M1M3 lowering failing to reach 0 position on the first try

Notebook containing data analysis for the event reported on [SITCOM-1171].

The M1M3 surrogate mirror had to be lowered twice on 15th January 2024, as the cell was getting ready for removal from TMA.

The first M1M3 lowering started around 12:17:45 UTC. The mirror's position as measured by the laser tracker was reported 10mm off in the Y direction once lowered. The surrogate mirror was raised and lowered again, starting around 12:29:18 UTC. The second try was successful, with the M1M3 being lowered correctly.


## Goal:
---

The goal is to identify what might cause the problem and fix the code so the lowering of the M1M3 at the zenith will always result in the mirror resting at the same position.


## Methodology
---
We will query data for different telemetry to analyze what might have caused the observed offset.

## Results
---
The offset reported by the laser tracker seems to not be real. IMS data and Hardpoint actuator data do not show the reported offset.

[SITCOM-1171]: https://jira.lsstcorp.org/browse/SITCOM-1171


## Preparing the Notebook

----

### Imports

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import astropy.units as u
from astropy.time import Time, TimeDelta


from lsst.ts.xml.enums.MTM1M3 import DetailedStates
from lsst.sitcom import vandv


## Helper function and classes

In [ ]:


class RaisingLoweringEvent:
    """Class to handle the telemetry associated with a raising or lowering event
    """

    # Used to the define what is considered as a zenith position
    MIN_EL_ZENITH = 88
    MAX_EL_ZENITH = 92

    def __init__(self, t_start, t_end, event_type):
        """A rasing or lowering event is defined by a start and end time and a type
        of event.

        Parameters
        ----------
        t_start : Time.Time
            Start time of the event
        t_end : Time.Time
            End time of the event
        event_type : lsst.ts.xml.enums.MTM1M3.DetailedStates
            Type of event. Can be RAISING, LOWERING, RAISINGENGINEERING or LOWERINGENGINEERING.
        """

        self.t_start = t_start
        self.t_end = t_end

        event_type = int(event_type)
        self.event_type = DetailedStates(event_type)


    async def at_zenith(self):
        """Check if the event happened at zenith.

        Returns
        -------
        bool
            True if the event happened at zenith, False otherwise
        """
        # Query the elevation of the telescope during the event
        el = await self.get_elevation()
        # Check if the elevation is within the zenith range
        return el >= self.MIN_EL_ZENITH and el <= self.MAX_EL_ZENITH


    async def get_elevation(self):
        """Queries the mean elevation of the telescope during the event.
        It should be constant during the event.

        Returns
        -------
        float
            Elevation of the telescope during the event
        """
        # start and end time of the event
        t_start_el = self.t_start
        t_end_el = self.t_end

        # Query the elevation of the telescope during the raising/lowering event
        # from the EDF
        df_el = await client.select_time_series(
            "lsst.sal.MTM1M3.inclinometerData",
            "inclinometerAngle",
            t_start_el,
            t_end_el,
            )

        # find another way to verify data
        # This should mean that there is no available data for these dates
        if not "inclinometerAngle" in df_el:
            print("No inclinometer data")
            return None
        # returns the mean elevation during the event
        el = np.mean(df_el["inclinometerAngle"])
        return el


    async def query_data_after_event(self, client, table, columns, time_delta_sec=10):
        """Queries telemetry data from the EFD after the end of the event, using a
        given time window.

        Parameters
        ----------
        client : lsst_efd_client.EfdClient
            Client to interact with the EFD
        table : str
            Name of the table (topic) to query from the EFD
        columns : list of str or str
            List of columns (fields) to query from the table (topic)
        time_delta_sec : int, optional
            Time window to query data after the event is finished, in seconds,
            by default 10

        Returns
        -------
        pd.DataFrame
            Dataframe containing the telemetry data
        """

        # We want to query data from the end of the event to a given time window
        t_end = self.t_end + TimeDelta(time_delta_sec, format="sec")
        df = await client.select_time_series(
            table,
            columns,
            self.t_end,
            t_end)
        return df


    async def query_data_during_event(self, client, table, columns):
        """Queries telemetry data from the EFD during the event.

        Parameters
        ----------
        client : lsst_efd_client.EfdClient
            Client to interact with the EFD
        table : str
            Name of the table (topic) to query from the EFD
        columns : list of str or str
            List of columns (fields) to query from the table (topic)

        Returns
        -------
        pd.DataFrame
            Dataframe containing the telemetry data
        """
        df = await client.select_time_series(
            table,
            columns,
            self.t_start,
            self.t_end)
        return df



async def find_events(time_start, time_end, detailed_state=DetailedStates.LOWERING):
    """Given a time window, find all the events of a given type of state.
    e.g. all the lowering events, all the raising events, etc.

    Parameters
    ----------
    time_start : Time.Time
        Start time of the time window
    time_end : Time.Time
        End time of the time window
    detailed_state : lsst.ts.xml.enums.MTM1M3.DetailedStates, optional
        Detailed states to find in the time window, by default DetailedStates.LOWERING

    Returns
    -------
    pd.DataFrame
        Dataframe containing the detailed state events
    """
    state = int(detailed_state)

    query = (
        "SELECT detailedState "
        'FROM "efd"."autogen"."lsst.sal.MTM1M3.logevent_detailedState" '
        f"WHERE time > '{time_start.isot}+00:00' AND time < '{time_end.isot}+00:00' "
        f"AND detailedState = {state} "
        f"ORDER BY time ASC "
    )
    df_states = await client.influx_client.query(query)
    return df_states


async def find_next_state(timestamp):
    """Given a timestamp, find the next state of the MTM1M3 detailed state.

    Parameters
    ----------
    timestamp : Time.Time
        Timestamp to query the next state

    Returns
    -------
    tuple: Time.Time, lsst.ts.xml.enums.MTM1M3.DetailedStates
        Timestamp of the next state and the next state
    """

    # We use a small delta to make sure we get the next state
    timestamp += TimeDelta(0.1, format="sec")

    # Query the next state of the MTM1M3 detailed state
    query = (
    "SELECT detailedState "
    'FROM "efd"."autogen"."lsst.sal.MTM1M3.logevent_detailedState" '
    f"WHERE time > '{timestamp.isot}+00:00' ORDER BY time ASC LIMIT 1"
    )
    ret = await client.influx_client.query(query)

    # We should have only one result
    next_state = ret["detailedState"].iloc[0]
    timestamp_next_state = ret["detailedState"].index[0]

    return timestamp_next_state, next_state


async def find_finished_rising_lowering_events(time_start, time_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=False):
    """Find all the finished raising or lowering events in a given time window.
    This is needed because not all the events are finished when they are executed.
    e.g. A lowering event can stop at 50% mirror supported percent and be resumed later.
    We would like to only consider the events that are finished at 0% .

    Finished events are considered as the following:
    - Lowering event that finished at PARKED
    - LoweringEngineering event that finished at PARKEDENGINEERING
    - Raising event that finished at ACTIVE
    - RaisingEngineering event that finished at ACTIVEENGINEERING


    Parameters
    ----------
    time_start : Time.Time
        Start time of the time window
    time_end : Time.Time
        End time of the time window
    detailed_state : lsst.ts.xml.enums.MTM1M3.DetailedStates, optional
        Detailed states to find in the time window, by default DetailedStates.LOWERING,
    only_at_zenith : bool, optional
        If True, only finds events that are executed
        with the telescope pointing at the zenith, by default False

    Returns
    -------
    list of RaisingLoweringEvent
        List of finished raising or lowering events
    """
    # Find all the events of a given type of state in the time window
    df_states = await find_events(time_start, time_end, detailed_state=detailed_state)

    events = []


    for event in df_states.iterrows():
        # The current state of the event
        current_state = event[1].iloc[0]
        timestamp = Time(event[0])
        # Find the next state of the event
        timestamp_next_state, next_state = await find_next_state(timestamp)

        event_finished = False
        # Check if the event is finished
        if current_state == DetailedStates.LOWERING and next_state == DetailedStates.PARKED:
            event_finished = True
        elif current_state == DetailedStates.LOWERINGENGINEERING and next_state == DetailedStates.PARKEDENGINEERING:
            event_finished = True
        elif current_state == DetailedStates.RAISING and next_state == DetailedStates.ACTIVE:
            event_finished = True
        elif current_state == DetailedStates.RAISINGENGINEERING and next_state == DetailedStates.ACTIVEENGINEERING:
            event_finished = True
        else:
            pass
            #print(current_state, next_state)

        if event_finished:
            # We create a rasing lowering event object
            event_obj = RaisingLoweringEvent(Time(timestamp), Time(timestamp_next_state), current_state)
            # Filter by events at zenith
            if (only_at_zenith and await event_obj.at_zenith()) or not only_at_zenith:
                events.append(event_obj)

    return events



async def plot_telemetry_after_finished_event(client, events_list, table, column, time_delta_sec=20,
                                              unit_column=None, unit_plot=None, y_min=None, y_max=None,
                                              fig=None, axs=None, save_plot=True, plot_filename=None, save_path=".",
                                             highlight_timestamp=None):
    """Plots the telemetry data from the EFD after the end of a list of events.
    All events are plotted on the same plot and aligned to the time of the end
    of the events.
    Two plots axis are created. The left one shows a time series of the telemetry
    after the definition of the end of the events. The right one shows an histogram
    of the distribution of the telemetry values at the end of the events.


    Parameters
    ----------
    client : lsst_efd_client.EfdClient
        Client to interact with the EFD
    events_list : list of RaisingLoweringEvent
        List of events to plot the telemetry data after the end of the event
    table : str
        Name of the table (topic) to query from the EFD
    column : str
        Name of the column (field) to query from the table (topic)
    time_delta_sec : int, optional
        Length of the time window to query and plot after the end of the events, by default 20
    unit_column : astropy.units, optional
        Default units of the column. by default None
    unit_plot : astropy.units, optional
       Unit to plot. Data are trasformed to this unit if possible (from m to mm)
       and this will be displayed in the plot labels, by default None.
    y_min : float, optional
       Min y axis value, by default None
    y_max : float, optional
        Max y axis value, by default None
    fig : matplotlib.figure.Figure, optional
        Figure used for plotting, by default None. If None, a new figure is created.
    axs : list of matplotlib.axes.Axes (len 2), optional
        List of axes to make the plots, by default None. If None, new axes are created.
    save_plot : bool, optional
        Whether the plot is saved to disk or not. by default True
    plot_filename : str, optional
        Plot filename. Overwrites default filename, by default None. If None
        the default filename is used.
    save_path : str, optional
        Path where the plots are saved to, by default "."
    highlight_timestamp : Time.Time, optional
        If an event occurred at this exact time, it will be highlighted in the plots
        , by default None.

    Returns
    -------
    fig, axs
        Figure and axes used for the plot. Useful if the user wants to modify the plot.

    Raises
    ------
    ValueError
        If the number of passed axes is not 2
    """

    # Creates the plots if not provided
    if fig is None or axs is None:
        fig, axs = plt.subplots(1, 2, sharey=True, figsize=(10, 5))
    if len(axs) != 2:
        raise ValueError(f"Axes should be a list of 2 axes objetcs")

    ax = axs[0]

    # Factor for unit conversion
    factor = 1
    if unit_column is not None and unit_plot is not None:
        factor = (1*unit_column).to_value(unit_plot)

    h_line_value = None
    end_values = []
    for event in events_list:
        # Query the telemetry data after the end of the event
        df = await event.query_data_after_event(client, table, column, time_delta_sec=time_delta_sec)
        # Time since the end of the event
        sec = (df.index - df.index[0])/np.timedelta64(1, "s")
        # Quantity to plot, transformed to the desired unit
        quantity = np.array(df[column]*factor)
        # Plot parameters
        plot_kwargs = {"c": "k", "alpha":0.5, "linewidth":1, "label": None}
        if highlight_timestamp and event.t_start == highlight_timestamp:
            # Highlight the event if it occurred at the exact time
            plot_kwargs["c"] = "r"
            plot_kwargs["alpha"] = 1
            plot_kwargs["label"] = f"Event on {highlight_timestamp}"
            h_line_value = quantity[-1]
        # plot the telemetry data
        ax.plot(sec, quantity, zorder=len(events_list), **plot_kwargs)
        # Store the last value of the telemetry data
        end_values.append(quantity[-1])

    ax.set_xlabel("Time since end of event (s)")
    unit_string = ""
    if unit_column is not None and unit_plot is not None:
        unit_string = f"[{unit_plot}]"

    label = f"{table}\n{column} {unit_string}"
    ax.set_ylabel(label)
    if y_min is not None and y_max is not None:
        ax.set_ylim(y_min, y_max)
    ax.legend()

    # in the right plot, we plot an histogram of the telemetry data at the end of the events
    ax = axs[1]
    bins = 100
    if y_min is not None and y_max is not None:
        bins = np.linspace(y_min, y_max, bins)
    ax.hist(end_values, bins=bins, orientation="horizontal")
    if h_line_value:
        pass
        #ax.axhline(h_line_value, c='r', label=f"Event on {highlight_timestamp}")
    ax.set_xlabel("counts")
    events_types = set([event.event_type.name for event in events_list])
    events = "-".join(events_types)
    ax.legend()
    fig.suptitle(events)

    fig.tight_layout()

    if save_plot:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        if plot_filename is None:
            # Default filename
            plot_filename = f"{column}_{table}_after_end_of_event_{events}.png"

        plot_filename = os.path.join(save_path, plot_filename)
        fig.savefig(plot_filename, dpi=200)

    return fig, ax

async def plot_telemetry_while_moving(client, events_list, table, column, unit_column=None, unit_plot=None, y_min=None, y_max=None, fig=None, ax=None,
                                     save_plot=True, save_path=".", plot_filename=None, highlight_timestamp=None):
    """Plots the telemetry data from the EFD during a list of events.

    Parameters
    ----------
    client : lsst_efd_client.EfdClient
        Client to interact with the EFD
    events_list : list of RaisingLoweringEvent
        List of events to plot the telemetry data after the end of the event
    table : str
        Name of the table (topic) to query from the EFD
    column : str
        Name of the column (field) to query from the table (topic)
    unit_column : astropy.units, optional
        Default units of the column. by default None
    unit_plot : astropy.units, optional
       Unit to plot. Data are trasformed to this unit if possible (from m to mm)
       and this will be displayed in the plot labels, by default None.
    y_min : float, optional
       Min y axis value, by default None
    y_max : float, optional
        Max y axis value, by default None
    fig : matplotlib.figure.Figure, optional
        Figure used for plotting, by default None. If None, a new figure is created.
    ax : matplotlib.axes.Axes (len 2), optional
        Ax to plot on. If None, new axes are created.
    save_plot : bool, optional
        Whether the plot is saved to disk or not. by default True
    plot_filename : str, optional
        Plot filename. Overwrites default filename, by default None. If None
        the default filename is used.
    save_path : str, optional
        Path where the plots are saved to, by default "."
    highlight_timestamp : Time.Time, optional
        If an event occurred at this exact time, it will be highlighted in the plots
        , by default None.

    Returns
    -------
    fig, ax
        Figure and axes used for the plot. Useful if the user wants to modify the plot.

    """

    # Creates the plots if not provided
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))

    # Factor for unit conversion
    factor = 1
    if unit_column is not None and unit_plot is not None:
        factor = (1*unit_column).to_value(unit_plot)

    for event in events_list:
        # Query the telemetry data during the event
        df = await event.query_data_during_event(client, table, column)
        # Time since the end of the event
        sec = (df.index - df.index[-1])/np.timedelta64(1, "s")
        # Quantity to plot, transformed to the desired unit
        quantity = np.array(df[column]*factor)

        # Plot parameters
        plot_kwargs = {"c": "k", "alpha":0.5, "linewidth":1, "label": None}
        if highlight_timestamp and event.t_start == highlight_timestamp:
            # Highlight the event if it occurred at the exact time
            plot_kwargs["c"] = "r"
            plot_kwargs["alpha"] = 1
            plot_kwargs["label"] = f"Event on {highlight_timestamp}"
        # plot the telemetry data
        ax.plot(sec, quantity, zorder=len(events_list), **plot_kwargs)


    ax.set_xlabel("Time since end of event (s)")
    unit_string = ""
    if unit_column is not None and unit_plot is not None:
        unit_string = f"[{unit_plot}]"

    label = f"{table}\n{column} {unit_string}"
    ax.set_ylabel(label)
    if y_min is not None and y_max is not None:
        ax.set_ylim(y_min, y_max)

    events_types = set([event.event_type.name for event in events_list])
    events = "-".join(events_types)
    fig.suptitle(events)
    ax.legend()
    fig.tight_layout()

    if save_plot:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        if plot_filename is None:
            # Default filename
            plot_filename = f"{column}_{table}_during_event_{events}.png"

        plot_filename = os.path.join(save_path, plot_filename)
        fig.savefig(plot_filename, dpi=200)

    return fig, ax


## Initial Analysis

We will start by querying lowering profiles for the weightSupportedPercent data
to verify we are actually getting finished lowering events.

In [ ]:
# Create the EFD client
client = vandv.efd.create_efd_client()

# This is the timestamp of the failed lowering event
t = np.datetime64("2024-01-15 12:17:16.272362+00:00")
t = Time(t)


# Query data from 1 year ago to now
t_start = Time("2023-07-01T00:00", format="isot", scale="utc")
t_end = Time("2024-02-28T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)

# Plot the supported percent profiles during lowering events
table = "lsst.sal.MTM1M3.logevent_raisingLoweringInfo"
column = "weightSupportedPercent"
await plot_telemetry_while_moving(client, lowering_events,  table, column, highlight_timestamp=t, save_plot=False)
plt.show()

## Lowering Profiles

The first thing we notice is that there is a difference in the lowering speeds. There seem to be two groups of lowering profiles. Manual examination shows that before and after 2024-01-11 the profiles are differents, indicating a change in the configuration. 


In [ ]:
# We plot the lowering profiles before and after the mentioned date
t_start = Time("2023-07-01T00:00", format="isot", scale="utc")
t_end = Time("2024-01-11T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)

# Plot the supported percent profiles during lowering events
table = "lsst.sal.MTM1M3.logevent_raisingLoweringInfo"
column = "weightSupportedPercent"
await plot_telemetry_while_moving(client, lowering_events,  table, column, highlight_timestamp=t, save_plot=False)

t_start = Time("2024-01-11T00:00", format="isot", scale="utc")
t_end = Time("2024-02-28T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)
await plot_telemetry_while_moving(client, lowering_events,  table, column, highlight_timestamp=t, save_plot=False)


## Position telemetry analysis

Now that we know that the lowering speed changed on a given date. For a fair comparison, we plot telemetry data before this date and after this date in different plots.

The telemetry we plot here is 

- lsst.sal.MTM1M3.imsData: "xPosition", "yPosition", "zPosition", "xRotation", "yRotation", "zRotation"
- lsst.sal.MTM1M3.hardpointActuatorData: "xPosition", "yPosition", "zPosition", "xRotation", "yRotation", "zRotation", "measuredForce0", "measuredForce1", "measuredForce2", "measuredForce3", "measuredForce4", "measuredForce5"
- sst.sal.MTM1M3.logevent_raisingLoweringInfo: "weightSupportedPercent"

Although the most relevant quantities are the yPosition from the IMS and the hardpoint data, as the 10 mm offset was reported for the y position.

Several plots will be created in directories named after the different start-end dates.

**The execution of the cells below can take several minutes, be aware of that**

In [ ]:
plots_dict = {'lsst.sal.MTM1M3.imsData': {"columns": ["xPosition", "yPosition", "zPosition", "xRotation", "yRotation", "zRotation"],
                                          "units_column":[u.m, u.m, u.m, u.deg, u.deg, u.deg],
                                          "units_plot":[u.mm, u.mm, u.mm, u.deg, u.deg, u.deg] },
              'lsst.sal.MTM1M3.hardpointActuatorData':  {"columns": ["xPosition", "yPosition", "zPosition", "xRotation", "yRotation", "zRotation", "measuredForce0", "measuredForce1", "measuredForce2", "measuredForce3", "measuredForce4", "measuredForce5"],
                                                                  "units_column":[u.m, u.m, u.m, u.deg, u.deg, u.deg, u.N, u.N, u.N, u.N, u.N, u.N],
                                                                 "units_plot":[u.mm, u.mm, u.mm, u.deg, u.deg, u.deg, u.N, u.N, u.N, u.N, u.N, u.N] },
              'lsst.sal.MTM1M3.logevent_raisingLoweringInfo': {"columns": ["weightSupportedPercent"],
                                                              "units_column":[None],
                                                               "units_plot":[None] },

             }


# Events in first configuration
t_start = Time("2023-07-01T00:00", format="isot", scale="utc")
t_end = Time("2024-01-11T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)


#raising_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISINGENGINEERING, only_at_zenith=True)
#raising_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISING, only_at_zenith=True)
#raising_events.extend(raising_events_eng)

save_path = f"{t_start.datetime.strftime('%d-%m-%Y')}_{t_end.datetime.strftime('%d-%m-%Y')}"

for table in plots_dict:
    columns = plots_dict[table]["columns"]
    units_columns = plots_dict[table]["units_column"]
    units_plots = plots_dict[table]["units_plot"]
    for column, unit_column, unit_plot in zip(columns, units_columns, units_plots):
        print(f"making plots for {table} {column}")
        await plot_telemetry_while_moving(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        #await plot_telemetry_while_moving(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)

        if column == "weightSupportedPercent":
            continue
        await plot_telemetry_after_finished_event(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        #await plot_telemetry_after_finished_event(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)


In [ ]:
# Events in second configuration
t_start = Time("2024-01-11T00:00", format="isot", scale="utc")
t_end = Time("2024-02-28T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)


#raising_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISINGENGINEERING, only_at_zenith=True)
#raising_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISING, only_at_zenith=True)
#raising_events.extend(raising_events_eng)
save_path = f"{t_start.datetime.strftime('%d-%m-%Y')}_{t_end.datetime.strftime('%d-%m-%Y')}"

for table in plots_dict:
    columns = plots_dict[table]["columns"]
    units_columns = plots_dict[table]["units_column"]
    units_plots = plots_dict[table]["units_plot"]
    for column, unit_column, unit_plot in zip(columns, units_columns, units_plots):
        print(f"making plots for {table} {column}")
        await plot_telemetry_while_moving(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        #await plot_telemetry_while_moving(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        if column == "weightSupportedPercent":
            continue
        #await plot_telemetry_after_finished_event(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        await plot_telemetry_after_finished_event(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)


In [ ]:
# Events including both configurations

t_start = Time("2023-07-01T00:00", format="isot", scale="utc")
t_end = Time("2024-02-28T00:00", format="isot", scale="utc")
lowering_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERINGENGINEERING, only_at_zenith=True)
lowering_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.LOWERING, only_at_zenith=True)
lowering_events.extend(lowering_events_eng)


#raising_events_eng = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISINGENGINEERING, only_at_zenith=True)
#raising_events = await find_finished_rising_lowering_events(t_start, t_end, detailed_state=DetailedStates.RAISING, only_at_zenith=True)
#raising_events.extend(raising_events_eng)

save_path = f"{t_start.datetime.strftime('%d-%m-%Y')}_{t_end.datetime.strftime('%d-%m-%Y')}"

for table in plots_dict:
    columns = plots_dict[table]["columns"]
    units_columns = plots_dict[table]["units_column"]
    units_plots = plots_dict[table]["units_plot"]
    for column, unit_column, unit_plot in zip(columns, units_columns, units_plots):
        print(f"making plots for {table} {column}")
        await plot_telemetry_while_moving(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        #await plot_telemetry_while_moving(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)

        if column == "weightSupportedPercent":
            continue
        await plot_telemetry_after_finished_event(client, lowering_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)
        #await plot_telemetry_after_finished_event(client, raising_events,  table, column, unit_column=unit_column, unit_plot=unit_plot, highlight_timestamp=t, save_path=save_path)


## Results

We are paying special attention to the final profiles and values for the lowering events in the yPosition, and considering the systematic offset between IMS and hardpoint ([SITCOM-760]). We find that the final position for the event reported in [SITCOM-1171] is not as off as the reported value of 10 mm, and is similar to the final position of other lowering events within 1 mm. This can be observed in the plots for just the second lowering configuration (since 11-01-2024) and both configurations (since 11-07-2023 to 28-02-2024). 

Important differences in other telemetries are also not observed, neither in x, y, or z positions from IMS and Hardpoint data nor in the measured forces in the hardpoints.

As a preliminary conclusion, we find that the reported offset in the mirror position for the lowering event on 15th January 2024 around 12:17:45 UTC did not happen, or at least is not registered in any telemetry available in the EDF. A possible alternative is that the laser tracker could have reported a wrong measurement. More investigation on this side is needed.


[SITCOM-1171]: https://jira.lsstcorp.org/browse/SITCOM-1171
[SITCOM-760]: https://jira.lsstcorp.org/browse/SITCOM-760